In [11]:
import tkinter as tk
import mysql.connector

# Connect to the MySQL database
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="rku_attendence_admin"
)

# Create the main window
root = tk.Tk()

# Set the window title and size
root.title("RKU Student Attendance System")
root.geometry("500x400")

# Add a label with the university name
label = tk.Label(root, text="Welcome to RKU Attendance System", font=("Arial", 20))
label.pack(pady=20)

# Add a label and entry for username
username_label = tk.Label(root, text="Username", font=("Arial", 14))
username_label.pack()
username_entry = tk.Entry(root, font=("Arial", 14))
username_entry.pack()

# Add a label and entry for password
password_label = tk.Label(root, text="Password", font=("Arial", 14))
password_label.pack()
password_entry = tk.Entry(root, show="*", font=("Arial", 14))
password_entry.pack()

# Add a button to log in as admin
def login():
    # Retrieve the values entered in the username and password entry fields
    username = username_entry.get()
    password = password_entry.get()

    # Query the MySQL database to check if the username and password match an admin account
    cursor = db.cursor()
    query = "SELECT * FROM admin WHERE username = %s AND password = %s"
    cursor.execute(query, (username, password))
    result = cursor.fetchone()

    if result:
        status_label.config(text="Login successful", fg="green")
    else:
        status_label.config(text="Login failed", fg="red")

login_button = tk.Button(root, text="Login as Admin", font=("Arial", 16), width=20, height=2, command=login)
login_button.pack(pady=10)

# Add a button to register as admin
def register():
    # Retrieve the values entered in the username and password entry fields
    username = username_entry.get()
    password = password_entry.get()

    # Insert a new admin account into the MySQL database
    cursor = db.cursor()
    query = "INSERT INTO admin (username, password) VALUES (%s, %s)"
    cursor.execute(query, (username, password))
    db.commit()

    status_label.config(text="Registration successful", fg="green")

register_button = tk.Button(root, text="Register as Admin", font=("Arial", 16), width=20, height=2, command=register)
register_button.pack(pady=10)

# Add a label to display the login/registration status
status_label = tk.Label(root, text="", font=("Arial", 14))
status_label.pack(pady=10)

# Add a button to reset the username and password entry fields
def reset():
    username_entry.delete(0, tk.END)
    password_entry.delete(0, tk.END)

reset_button = tk.Button(root, text="Reset", font=("Arial", 12), width=10, command=reset)
reset_button.pack(pady=10)

# Run the main event loop
root.mainloop()

# Close the database connection
db.close()


In [14]:
import tkinter as tk
from tkinter import ttk
import mysql.connector
from tkinter import messagebox

class StudentRegistrationForm(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.title("Student Registration Form")
        
        # Connect to database
        self.db = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="rku_attendence_admin"
        )
        
        
        # Create label and entry for IT department student
        self.it_label = tk.Label(master, text="IT Department")
        self.it_label.grid(row=0, column=0)
        self.it_name_label = tk.Label(master, text="Name:")
        self.it_name_label.grid(row=1, column=0)
        self.it_name_entry = tk.Entry(master)
        self.it_name_entry.grid(row=1, column=1)
        self.it_roll_label = tk.Label(master, text="Roll No.:")
        self.it_roll_label.grid(row=2, column=0)
        self.it_roll_entry = tk.Entry(master)
        self.it_roll_entry.grid(row=2, column=1)

        # Create label and entry for CE department student
        self.ce_label = tk.Label(master, text="CE Department")
        self.ce_label.grid(row=0, column=2)
        self.ce_name_label = tk.Label(master, text="Name:")
        self.ce_name_label.grid(row=1, column=2)
        self.ce_name_entry = tk.Entry(master)
        self.ce_name_entry.grid(row=1, column=3)
        self.ce_roll_label = tk.Label(master, text="Roll No.:")
        self.ce_roll_label.grid(row=2, column=2)
        self.ce_roll_entry = tk.Entry(master)
        self.ce_roll_entry.grid(row=2, column=3)
        
        # Create table to display registered students
        self.table_label = tk.Label(master, text="Registered Students")
        self.table_label.grid(row=3, column=0, columnspan=4)
        self.table = ttk.Treeview(master, columns=("name", "roll"))
        self.table.heading("#0", text="Department")
        self.table.heading("name", text="Name")
        self.table.heading("roll", text="Roll No.")
        self.table.column("#0", width=100)
        self.table.column("name", width=150)
        self.table.column("roll", width=100)
        self.table.grid(row=4, column=0, columnspan=4)
        
        # Create button to register students
        self.register_button = tk.Button(master, text="Register", command=self.register_students)
        self.register_button.grid(row=5, column=0, columnspan=4)
        
     
 

    def register_students(self):
        it_name = self.it_name_entry.get()
        it_roll = self.it_roll_entry.get()
        ce_name = self.ce_name_entry.get()
        ce_roll = self.ce_roll_entry.get()

    # Check if IT student details are entered
        if it_name and it_roll:
            cursor = self.db.cursor()
            sql = "INSERT INTO it_students (name, roll) VALUES (%s, %s)"
            values = (it_name, it_roll)
            cursor.execute(sql, values)
            self.db.commit()
            messagebox.showinfo("Success", "IT student registered successfully!")
    
        # Check if CE student details are entered
        if ce_name and ce_roll:
            cursor = self.db.cursor()
            sql = "INSERT INTO ce_students (name, roll) VALUES (%s, %s)"
            values = (ce_name, ce_roll)
            cursor.execute(sql, values)
            self.db.commit()
            messagebox.showinfo("Success", "CE student registered successfully!")
        
        
        # Clear the existing rows in the table
        for row in self.table.get_children():
            self.table.delete(row)
    
        # Fetch the updated data from the database to populate the table again
        cursor = self.db.cursor()
        sql = "SELECT name, roll FROM it_students"
        cursor.execute(sql)
        it_students = cursor.fetchall()
    
        sql = "SELECT name, roll FROM ce_students"
        cursor.execute(sql)
        ce_students = cursor.fetchall()
    
        # Add the IT students to the table
        for student in it_students:
            self.table.insert("", "end", text="IT", values=student)
    
        # Add the CE students to the table
        for student in ce_students:
            self.table.insert("", "end", text="CE", values=student)
     
    if __name__ == "__main__":
        root = tk.Tk()
        app = StudentRegistrationForm(master=root)
        app.mainloop()
